In [2]:
# Set up: Install required packages, etc.
# import Pkg; Pkg.add("DataFrames")
# import Pkg; Pkg.add("CSV")
# import Pkg; Pkg.add("StatFiles")
# import Pkg; Pkg.add("GLM")
# import Pkg; Pkg.add("CovarianceMatrices")
import Pkg; Pkg.add("ShiftedArrays")

   Resolving package versions...
    Updating `~/.julia/environments/v1.11/Project.toml`
  [1277b4bf] + ShiftedArrays v2.0.0
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [ ]:
# Load and clean data
using DataFrames, CSV, StatFiles
us_data = DataFrame(load("us_data_updated.dta"))

In [4]:
using DataFrames, StatFiles, ShiftedArrays

# --- Load SPF data ---
df = DataFrame(load("us_data_updated.dta"))

# Keep only SPF and inflation
df_spf = filter(r -> r.forecsource == "spf" && r.forvar == "pi", df)

# Keep only relevant variables
df_spf = select(df_spf, [:timeseq, :year, :quarter, :fehor, :ind_fe, :ind_fr, :ind_for])

# --- Reshape to wide (like reshape wide in Stata) ---
df_fe  = unstack(df_spf, :fehor, :ind_fe)   # columns ind_fe0, ind_fe1, ...
df_fr  = unstack(df_spf, :fehor, :ind_fr)
df_for = unstack(df_spf, :fehor, :ind_for)

# Merge reshaped pieces
df_wide = innerjoin(df_fe, df_fr, on=[:timeseq, :year, :quarter])
df_wide = innerjoin(df_wide, df_for, on=[:timeseq, :year, :quarter])

# --- Generate compounded forecasts ---
df_wide.ind_for_30 = ((df_wide.ind_for0 ./ 400 .+ 1) .* 
                      (df_wide.ind_for1 ./ 400 .+ 1) .* 
                      (df_wide.ind_for2 ./ 400 .+ 1) .* 
                      (df_wide.ind_for3 ./ 400 .+ 1) .- 1) .* 100

df_wide.ind_for_41 = ((df_wide.ind_for1 ./ 400 .+ 1) .* 
                      (df_wide.ind_for2 ./ 400 .+ 1) .* 
                      (df_wide.ind_for3 ./ 400 .+ 1) .* 
                      (df_wide.ind_for4 ./ 400 .+ 1) .- 1) .* 100

df_wide.ind_fe_30  = ((df_wide.ind_fe0 ./ 400 .+ 1) .* 
                      (df_wide.ind_fe1 ./ 400 .+ 1) .* 
                      (df_wide.ind_fe2 ./ 400 .+ 1) .* 
                      (df_wide.ind_fe3 ./ 400 .+ 1) .- 1) .* 100

# --- Forecast revision variable ---
df_wide.fr_30_41 = df_wide.ind_for_30 .- lag(df_wide.ind_for_41, 1)

# --- Merge macro data ---
macro = DataFrame(load("Macro-data-FRED.dta"))
df_merged = innerjoin(df_wide, macro, on=[:year, :quarter])

# --- Create lagged macro controls ---
df_merged.L_inf    = lag(df_merged.inflation_realtime, 1)
df_merged.L_tb3ms  = lag(df_merged.tb3ms, 1)
df_merged.L_unrate = lag(df_merged.unrate, 1)

# Oil price variable
df_merged.oilP = coalesce.(df_merged.oilprice, df_merged.mcoilwtico)

# Lagged oil price differences
df_merged.LD_oilP = log.(lag(df_merged.oilP, 1) ./ lag(df_merged.oilP, 2))

# Final variable used later
df_merged.L_inf4 = df_merged.ind_for_30

# df_merged is now your cleaned dataset ready for regressions



LoadError: ArgumentError: Duplicate variable names: :ind_for, :0, :1, :2, :3 and :4. Pass makeunique=true to make them unique using a suffix automatically.